In [55]:
import re
import random
import itertools

In [72]:
class Simple:
    def __init__(self, *books):

        with open(books, 'r', encoding='utf-8') as f:
            lines = f.readlines()

s = Simple("alice.txt")

s = Simple("hi", "there", 40)

TypeError: expected str, bytes or os.PathLike object, not tuple

In [75]:
import re
import random

class BookReader:

    def __init__(self, *books):
        chunk_size = 25
        lines = []
        for book in books:
            with open(book, 'r', encoding='utf-8') as f:
                lines += f.readlines()

        self.all_lines = []
        self.train = []
        self.dev   = []
        self.test  = []

        self.chunk = []
        has_newline = False
        for line in lines:
            if line == '\n' and has_newline:
                has_newline = False
                continue
            else:
                has_newline = True
            line = re.sub(r'(\n)+', '\n', line)
            self.all_lines += line
            self.chunk += line

            if len(self.chunk) % chunk_size == 0:
                match random.randint(0, 10):
                    case 0:
                        self.test += self.chunk
                    case 1:
                        self.dev += self.chunk
                    case _:
                        self.train += self.chunk

                self.chunk = []
            
        self.pattern = r'[^\d+a-zA-Z \n(1)?!]'
        
        self.text = re.sub(self.pattern, ' ', "".join(self.all_lines)).lower()
        self.train = re.sub(self.pattern, ' ', "".join(self.train)).lower()
        self.dev = re.sub(self.pattern, ' ', "".join(self.dev)).lower()
        self.test = re.sub(self.pattern, ' ', "".join(self.test)).lower()

        # here are all the unique characters that occur in this text
        self.chars = sorted(list(set(self.text)))
        
        self.vocab_size = len(self.chars)
        # create a mapping from characters to integers
        self.stoi = { ch:i for i,ch in enumerate(self.chars) }
        self.itos = { i:ch for i,ch in enumerate(self.chars) }
        self.encode = lambda s: [self.stoi[c] for c in s] # encoder: take a string, output a list of integers
        self.decode = lambda l: ''.join([self.itos[i] for i in l]) # decoder: take a list of integers, output a string

        self.data = [self.encode(self.train), self.encode(self.dev), self.encode(self.test)]
        
alice = BookReader("alice.txt")

print(alice.vocab_size, len(alice.data[0]), len(alice.data[1]), len(alice.data[2]))

42 114300 28450 18875


In [43]:
remove = "1 remove\n\n\n"
re.sub(r'(\n)+', '\n', remove).lower()

'rremove\n'

In [11]:
import torch

In [54]:
train_data = torch.tensor(alice.data[0][:2000])
print(train_data)
alice.decode(train_data.tolist())

tensor([ 1, 18, 23,  ..., 20,  1, 16])


' chapter i \ndown the rabbit hole\n\n\nalice was beginning to get very tired of sitting by her sister on the\nbank  and of having nothing to do  once or twice she had peeped into\nthe book her sister was reading  but it had no pictures or\nconversations in it   and what is the use of a book   thought alice\n without pictures or conversations? \n\nso she was considering in her own mind (as well as she could  for the\nhot day made her feel very sleepy and stupid)  whether the pleasure of\nmaking a daisy chain would be worth the trouble of getting up and\npicking the daisies  when suddenly a white rabbit with pink eyes ran\nclose by her \n\nthere was nothing so  very  remarkable in that  nor did alice think it\nso  very  much out of the way to hear the rabbit say to itself   oh\ndear! oh dear! i shall be late!  (when she thought it over afterwards \nit occurred to her that she ought to have wondered at this  but at the\ntime it all seemed quite natural)  but when the rabbit actually  too

In [15]:
# data loading
def get_batch(data, sequence_length=4, batch_size=5):
    # generate a small batch of data of inputs x and targets y
    ix = torch.randint(len(data) - sequence_length, (batch_size,))
    b = torch.stack([data[i:i+sequence_length+1] for i in ix])
    return b

get_batch(train_data)

tensor([[25,  1, 32, 18, 18],
        [18, 17,  1, 33, 28],
        [ 1, 21, 18, 31,  1],
        [27,  0,  0, 17, 22],
        [18, 31,  1, 19, 18]])